In [2]:
import os, codecs, json, time
import pandas as pd
import numpy as np
import sklearn as sl
import matplotlib.pyplot as plt
% matplotlib inline

In [3]:
INPUT_FILE='data\\train.csv'
WASH_FLAG = False

In [4]:
st = time.time()
train_df = pd.read_csv(INPUT_FILE, nrows=1000)
print ("Reading", time.time()-st)
print (len(train_df))

Reading 0.243849515914917
1000


In [5]:
train_df.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [6]:
train_df.device[0]

'{"browser": "Chrome", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Windows", "operatingSystemVersion": "not available in demo dataset", "isMobile": false, "mobileDeviceBranding": "not available in demo dataset", "mobileDeviceModel": "not available in demo dataset", "mobileInputSelector": "not available in demo dataset", "mobileDeviceInfo": "not available in demo dataset", "mobileDeviceMarketingName": "not available in demo dataset", "flashVersion": "not available in demo dataset", "language": "not available in demo dataset", "screenColors": "not available in demo dataset", "screenResolution": "not available in demo dataset", "deviceCategory": "desktop"}'

In [7]:
train_df.geoNetwork[0]

'{"continent": "Asia", "subContinent": "Western Asia", "country": "Turkey", "region": "Izmir", "metro": "(not set)", "city": "Izmir", "cityId": "not available in demo dataset", "networkDomain": "ttnet.com.tr", "latitude": "not available in demo dataset", "longitude": "not available in demo dataset", "networkLocation": "not available in demo dataset"}'

In [8]:
train_df.totals[0]

'{"visits": "1", "hits": "1", "pageviews": "1", "bounces": "1", "newVisits": "1"}'

In [9]:
train_df.trafficSource[0]

'{"campaign": "(not set)", "source": "google", "medium": "organic", "keyword": "(not provided)", "adwordsClickInfo": {"criteriaParameters": "not available in demo dataset"}}'

In [10]:
train_df.describe()

,date,fullVisitorId,visitId,visitNumber,visitStartTime
count,1000.0,1.000000e+03,1.000000e+03,1000.000000,1.000000e+03
mean,20160902.0,4.545762e+18,1.472841e+09,2.035000,1.472841e+09
std,0.0,3.053412e+18,2.240422e+04,6.515384,2.240424e+04
min,20160902.0,2.953262e+15,1.472800e+09,1.000000,1.472800e+09
25%,20160902.0,1.647353e+18,1.472824e+09,1.000000,1.472824e+09
50%,20160902.0,4.418539e+18,1.472840e+09,1.000000,1.472840e+09
75%,20160902.0,7.256540e+18,1.472858e+09,1.000000,1.472858e+09
max,20160902.0,9.991882e+18,1.472886e+09,136.000000,1.472886e+09


As a result, we have to deal with fields written in JSON format.

In [11]:
from pandas.io.json import json_normalize
json_fields = ['device', 'geoNetwork', 'totals', 'trafficSource']
def flatten(flag=False):
    if not flag:
        return   
    df = pd.read_csv(INPUT_FILE, 
                     converters={column: json.loads for column in json_fields}, 
                     dtype={'fullVisitorId': 'str'})
    
    for col in json_fields:
        col_as_df = json_normalize(df[col])
        col_as_df.columns = [f"{col}.{subcol}" for subcol in col_as_df.columns]
        df = df.drop(col, axis=1).merge(col_as_df, right_index=True, left_index=True)
    df.to_csv('data\\f_train.csv')
flatten(WASH_FLAG)
train_df_f = pd.read_csv('data\\f_train.csv')

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
train_df_f.head()

,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


Browse the flattened file in Microsoft Excel, I see the values of colmns, especially for those text data. I decided to write lines of codes to automatically identicate how many categories they have. Text data can be view into two groups: plain string data like ID# and category data. Some colomns contains integer value can also be regarded as category data.

Real number data:
 - visitNumber
 - totals.hits
 - totals.pageviews
 - totals.transactionRevenue

Datetime data:
 - date
 - visitStartTime (TimeStamp)

Plain string data:
 - fullVisitorId
 - visitId
 - sessionId
 
Category data:
 - channelGrouping
 - socialEngagementType
 - device.browser
 - device.browserSize
 - device.browserVersion
 - device.deviceCategory
 - device.flashVersion
 - device.isMobile
 - device.language
 - device.mobileDeviceBranding
 - device.mobileDeviceInfo
 - device.mobileDeviceMarketingName
 - device.mobileDeviceModel
 - device.mobileInputSelector
 - device.operatingSystem
 - device.operatingSystemVersion
 - device.screenColors
 - device.screenResolution
 - geoNetwork.city
 - geoNetwork.cityId
 - geoNetwork.continent
 - geoNetwork.country
 - geoNetwork.latitude
 - geoNetwork.longitude
 - geoNetwork.metro
 - geoNetwork.networkDomain
 - geoNetwork.networkLocation
 - geoNetwork.region
 - geoNetwork.subContinent
 - totals.bounces
 - totals.newVisits
 - totals.transactionRevenue
 - totals.visits
 - trafficSource.adContent
 - trafficSource.adwordsClickInfo.adNetworkType
 - trafficSource.adwordsClickInfo.criteriaParameters
 - trafficSource.adwordsClickInfo.gclId
 - trafficSource.adwordsClickInfo.isVideoAd
 - trafficSource.adwordsClickInfo.page
 - trafficSource.adwordsClickInfo.slot
 - trafficSource.campaign
 - trafficSource.campaignCode
 - trafficSource.isTrueDirect
 - trafficSource.keyword
 - trafficSource.medium
 - trafficSource.referralPath
 - trafficSource.source

In [12]:
CATEGORY_COLUMNS = [
    'channelGrouping',
    'socialEngagementType',
    'device.browser',
    'device.browserSize',
    'device.browserVersion',
    'device.deviceCategory',
    'device.flashVersion',
    'device.isMobile',
    'device.language',
    'device.mobileDeviceBranding',
    'device.mobileDeviceInfo',
    'device.mobileDeviceMarketingName',
    'device.mobileDeviceModel',
    'device.mobileInputSelector',
    'device.operatingSystem',
    'device.operatingSystemVersion',
    'device.screenColors',
    'device.screenResolution',
    'geoNetwork.city',
    'geoNetwork.cityId',
    'geoNetwork.continent',
    'geoNetwork.country',
    'geoNetwork.latitude',
    'geoNetwork.longitude',
    'geoNetwork.metro',
    'geoNetwork.networkDomain',
    'geoNetwork.networkLocation',
    'geoNetwork.region',
    'geoNetwork.subContinent',
    'totals.bounces',
    'totals.newVisits',
    'totals.transactionRevenue',
    'totals.visits',
    'trafficSource.adContent',
    'trafficSource.adwordsClickInfo.adNetworkType',
    'trafficSource.adwordsClickInfo.criteriaParameters',
    'trafficSource.adwordsClickInfo.gclId',
    'trafficSource.adwordsClickInfo.isVideoAd',
    'trafficSource.adwordsClickInfo.page',
    'trafficSource.adwordsClickInfo.slot',
    'trafficSource.campaign',
    'trafficSource.campaignCode',
    'trafficSource.isTrueDirect',
    'trafficSource.keyword',
    'trafficSource.medium',
    'trafficSource.referralPath',
    'trafficSource.source',
]

In [ ]:
for col in CATEGORY_COLUMNS:
    print (col, len(set(train_df_f[col])))

channelGrouping 8
socialEngagementType 1
device.browser 54
device.browserSize 1
device.browserVersion 1
device.deviceCategory 3
device.flashVersion 1
device.isMobile 2
device.language 

In [7]:
set(train_df_f['device.mobileDeviceBranding'])

{'not available in demo dataset'}

From above we can observe that, category colmns listed below are only have one category value:
 - socialEngagementType
 - device.browerSize
 - device.flashVersion
 - device.language
 
Therefore, they can be totally removed from the dataset.

